# Heart Disease Prediction

## Dataset Cleaning

In [1]:
# IMPORT DATASET TRAIN AND TEST
import pandas as pd
import numpy as np
dataset = pd.read_csv('data/tubes2_HeartDisease_train.csv')
test_set = pd.read_csv('data/tubes2_HeartDisease_test.csv')

In [2]:
# DATASET ClEANING
# Column Dataset Train
## Column 4 = 46 kosong
## Column 5 = 24
## Column 6 = 78
## Column 7 = 1 <- nilainya jadiin 0
## Column 8 = 43
## Column 9 = 43 angina
## Column 10 = 48 ST depression
## Column 11 = 261 
## Column 12 = 513 BUANG
## Column 13 = 407 BUANG
import math
dataset = dataset.drop(columns=['Column12', 'Column13'], axis=1)
test_set = test_set.drop(columns=['Column12', 'Column13'])

data_len = len(dataset['Column7'])
for i in range(0, data_len):
    if dataset['Column7'][i] == '?' or math.isnan(float(dataset['Column7'][i])):
        dataset['Column7'][i] = '0'

d:\program files\python36\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
# Hapus row yang ?-nya lebih dari 5
#deleterow = []
#for i in range(0, 779):
#    count = 0
#    for j in range(0, 12):
#        if dataset.iloc[i][j] == '?':
#            count += 1
#    if count >= 5:
#        deleterow.append(i)
#print(deleterow)

#dataset = dataset.drop(deleterow)

In [ ]:
# Replace '?' data with mean value
dataset['Column4'][15]
avgList = []
for i in range(3, 12):
    sum = 0
    count = 0
    for j in range(0, 736):
        if dataset.iloc[j][i] != '?':
            sum += float(dataset.iloc[j][i])
            count += 1
    avg = sum/count
    avgList.append(round(avg))
col = 'Column'
for i in range(4, 12):
    column = col + str(i)
    for j in range(0, 736):
        if dataset[column][j] == '?':
            dataset[column][j] = avgList[i-4]
    for k in range(0, len(test_set[column])):
        if test_set[column][k] == '?':
            test_set[column][k] = avgList[i-4]

d:\program files\python36\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\program files\python36\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
# Dataset yang sudah dibersihin
dataset
#test_set

In [ ]:
from random import choice
import numpy as np
import math

def generate_fold(k, dataset):
    dataset_size = len(dataset)
    fold = [[] for i in range (k)]
    nums = [i for i in range(k)]
    sizes = [0 for i in range(k)]
    normal_size = math.floor(dataset_size / k)
    max_size = math.ceil(dataset_size / k)
    size = max_size
    max_size_counter = dataset_size % k
    
    for val in dataset:
        idx = choice(nums)
        fold[idx].append(val)
        sizes[idx] += 1
        
        if sizes[idx] == size:
            nums.remove(idx)
        
        if sizes[idx] == max_size:
            max_size_counter -= 1
            if max_size_counter == 0:
                size = normal_size
                
                temp = []
                for num in nums:
                    if sizes[num] == size:
                        temp.append(num)
                for t in temp:
                    nums.remove(t)
    return fold

def seperate(dataset):
    params = []
    lables = []
    datasize = len(dataset[0])
    for data in dataset:
        params.append(data[:datasize-1])
        lables.append(data[datasize-1])
    return params, lables

def parse_dataset(frame):
    dataset = []
    for index, row in frame.iterrows():
        dataset.append(row.values.tolist())
    return dataset

def get_trainingset(index, folds):
    training_set = []
    for i in range(len(folds)):
        if i != index:
            for data in folds[i]:
                training_set.append(data)
    return training_set

def pseudo_clean(dataset):
    cleaned = []
    idx = 0
    for data in dataset:
        row = []
        for val in data:
            row.append(float(val))
        cleaned.append(row)
        idx += 1
    return cleaned

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

def train(trainer, folds):
    classifiers = []
    for i in range (len(folds)):
        training_set = get_trainingset(i, folds)
        training_params, training_lables = seperate(training_set)
        classifiers.append(trainer.fit(training_params, training_lables))
    return classifiers

def train_naive_bayes(folds):
    return train(GaussianNB(), folds)

def train_decision_tree(folds):
    return train(tree.DecisionTreeClassifier(), folds)

def train_knn(neighbor, folds):
    return train(KNeighborsClassifier(neighbor), folds)

def train_mlp(folds):
    return train(MLPClassifier(learning_rate_init=0.01, max_iter=300), folds)

In [ ]:
dataset = parse_dataset(dataset)
dataset = pseudo_clean(dataset)
folds = generate_fold(10, dataset)

nb_models = train_naive_bayes(folds)
dt_models = train_decision_tree(folds)
knn_models = train_knn(5, folds)
mlp_models = train_mlp(folds)

In [ ]:
from sklearn.metrics import accuracy_score

def generate_accuracies(models):
    for i in range(len(models)):
        test_set = folds[i]
        test_params, test_lables = seperate(test_set)
        predictions = models[i].predict(test_params)
        accuracy = accuracy_score(np.array(test_lables), predictions)
        print(str(round(accuracy * 100, 2)) + '%')

In [ ]:
print("Naive Bayes Accuracies:")
generate_accuracies(nb_models)
print("Decision Tree Accuracies:")
generate_accuracies(dt_models)
print("K-Nearest Neighbors Accuracies:")
generate_accuracies(knn_models)
print("Multi-Layer Perceptron Accuracies:")
generate_accuracies(mlp_models)

In [ ]:
import graphviz 
dot_data = tree.export_graphviz(dt_models[0], out_file=None) 
graph = graphviz.Source(dot_data)  
graph 